# Kinematics, Dynamics, Visualization + Control

In [1]:
import numpy as np
from numpy import sin, cos
import matplotlib.pyplot as plt
import time
from scipy.integrate import solve_ivp
from sklearn.neighbors import NearestNeighbors
from functools import partial

from IPython.display import clear_output
import pdb
from utils import subsample

from scipy.optimize import fmin



np.set_printoptions(precision=4, suppress=True)
%load_ext autoreload
%autoreload 2

## Part 1: Standard Controller 

In [2]:
from ocp_sys import Biped_Casadi

#### Try solving and analyse result

In [3]:
%matplotlib qt

In [4]:
biped = Biped_Casadi()

#standard values with standard params
q0 = np.array([np.pi/9, -np.pi/9, 0.])
dq0 = np.array([0.,0.,8.])

#standard values with already stable limit cycle
# q0, dq0 = (np.array([-0.3035,  0.3608,  0.2487]), np.array([ 0.8319, -1.1396,  4.6439]))
# qT, dqT = (np.array([ 0.3623, -0.3046,  0.2487]), np.array([ 2.4543, -1.8658, -0.2872]))
# T = int(0.375/biped.dT)

# #standard values with already (almost) stable limit cycle for ilqr
q0, dq0 = (np.array([-0.3075,  0.3459,  0.2666]), np.array([ 0.8131, -1.1405,  5.0078]))
qT, dqT = (np.array([ 0.3623, -0.3046,  0.2487]), np.array([ 2.4543, -1.8658, -0.2872]))
#(array([-0.3025,  0.3241,  0.233 ]), array([ 0.8955, -1.4391,  4.6021]))
#(array([-0.3038,  0.3368,  0.2396]), array([ 0.8731, -1.3125,  4.6623]))
T = int(0.375/biped.dT)

In [7]:
sln = biped.solve_eqns(q0, dq0, 10, biped.control_parameters)
clear_output()

#biped.animate(sln, dt = 0.001)

In [8]:
res = biped.analyse(sln, biped.control_parameters, to_plot=True)

COT: 1812.72, Dist: 0.326, dT: 0.375


In [9]:
%matplotlib qt 
biped.animate(sln)

Real time factor:0.2907611684323586


0.2907611684323586

#### Try optimizing

In [8]:
def opt_func(opt_parameters, num_steps):
    print(num_steps)
    # extract parameters q0, dq0 and x
    q0 = opt_parameters[:3]
    dq0 = opt_parameters[3:6]
    x = opt_parameters[6:]

    # run simulation
    sln = biped.solve_eqns(q0, dq0, num_steps, x);
    results = biped.analyse(sln, x, False)

    # calculate metrics such as distance, mean velocity and cost of transport
    effort = results['effort']
    distance = np.sum(results['ds'])
    frequency = (results['fs'][-1])
    velocity = results['total_mean_velocity']
    CoT = results['cot']
    w1, w2, vd = .1, 0.1, 1.
    objective_value = w1*abs(vd - velocity) + w2*CoT
    #objective_value = -np.mean(results['ds']) #+ 0.0001*CoT

    if distance < 0 or CoT < 0 or velocity < 0:
        objective_value = 1000        
    print(objective_value)
    print(opt_parameters)
    return objective_value

### Observe the resulting initial and final configs